# Objectives 
* Determine if logistic regression can be done with a non-gigantic amount of variables, and if so, which variables
* Find patterns in the data and visualize them

First, we import the necessary libraries and get a first look at the data.

In [58]:
import pandas as pd
# probably import matplotlib and seaborn later

# + blue side lead, - red side lead, warding numbers wonky, timer variables in seconds

In [59]:
df = pd.read_csv('hackathon-riot-data/hackathon-riot-data.csv', sep=';')
pd.set_option('display.max_columns', None)

In [60]:
# Unnamed is a redundant column same thing as index, ID doesn't influence who wins, ward numbers are unreliable
# several columns like camps secured, number of drakes, scuttles, heralds, are all 0 and redundant anyways
df.drop(columns=['Unnamed: 0', 
                 'esportsPlatformId', 
                 'NbWardsPlacedBlue', 
                 'NbWardsPlacedRed', 
                 'NbControlWardsPlacedBlue', 
                 'NbControlWardsPlacedRed', 
                 'NbWardsKilledBlue', 
                 'NbWardsKilledRed', 
                 'NbControlWardsKilledBlue', 
                 'NbControlWardsKilledRed', 
                 'NbCampsSecuredBlue', 
                 'NbCampsSecuredRed',
                 'NbScuttlesBlue',
                 'NbScuttlesRed',
                 'NbRiftHeraldsBlue',
                 'NbRiftHeraldsRed', 
                 'NbDragonsBlue',
                 'NbDragonsRed',
                 'NbBaronsBlue',
                 'NbBaronsRed',
                 'NbEldersBlue',
                 'NbEldersRed',
                 'DragonSoulTimer',
                 'DragonSoulType',
                 'DragonSoulTaker'], inplace=True)

First, we need to check for missing values and address them appropriately, see if any can be replaced or if the row they belong to must be dropped.

In [61]:
col_list = df.columns.tolist()

for col in col_list:
    if df[col].isnull().sum() != 0:
        print(col.ljust(25), str(df[col].isnull().sum()).rjust(5))

gameDate                      3
gameVersion                   3
gameDuration                  3
OuterTopBlueTimer          3105
OuterMidBlueTimer          4976
OuterBotBlueTimer          2874
InnerTopBlueTimer         11699
InnerMidBlueTimer          8947
InnerBotBlueTimer         10211
BaseTopBlueTimer          15866
BaseMidBlueTimer          11583
BaseBotBlueTimer          14357
Nexus1MidBlueTimer        10208
Nexus2MidBlueTimer        10351
OuterTopRedTimer           2221
OuterMidRedTimer           3524
OuterBotRedTimer           2409
InnerTopRedTimer          10169
InnerMidRedTimer           7454
InnerBotRedTimer           9202
BaseTopRedTimer           15082
BaseMidRedTimer           10425
BaseBotRedTimer           13452
Nexus1MidRedTimer          9035
Nexus2MidRedTimer          9232
GoldTopBlue10                 3
GoldJgBlue10                  3
GoldMidBlue10                 3
GoldADBlue10                  3
GoldSupBlue10                 3
GoldTopRed10                  3
GoldJgRe

In [62]:
# can't use a row without the winner label since that's what is being predicted
missing_label_row = (df['winner'].isna() == True)
df.drop(index=df[missing_label_row].index, inplace=True)

# 3 random rows with a lot of NaNs
nan_rows = (df['gameDate'].isna() == True)
df.drop(index=df[nan_rows].index, inplace=True)
df.reset_index(drop=True, inplace=True)

In [63]:
# NaN represents a turret was never taken, so it can be replaced with the game duration
# doing this with a loop because of some setting copy error

for col in col_list:
    if df[col].isnull().sum() != 0:
        df[col].fillna(df['gameDuration'], inplace=True)

In [55]:
df

,gameDate,gameVersion,gameDuration,NbCampsStolenBlue,NbCampsStolenRed,NbTowersBlue,NbTowersRed,NbPlatesBlue,NbPlatesRed,OuterTopBlueTimer,OuterMidBlueTimer,OuterBotBlueTimer,InnerTopBlueTimer,InnerMidBlueTimer,InnerBotBlueTimer,BaseTopBlueTimer,BaseMidBlueTimer,BaseBotBlueTimer,Nexus1MidBlueTimer,Nexus2MidBlueTimer,OuterTopRedTimer,OuterMidRedTimer,OuterBotRedTimer,InnerTopRedTimer,InnerMidRedTimer,InnerBotRedTimer,BaseTopRedTimer,BaseMidRedTimer,BaseBotRedTimer,Nexus1MidRedTimer,Nexus2MidRedTimer,GoldTopBlue10,GoldJgBlue10,GoldMidBlue10,GoldADBlue10,GoldSupBlue10,GoldTopRed10,GoldJgRed10,GoldMidRed10,GoldADRed10,GoldSupRed10,GoldDiff10Top,GoldDiff10Jg,GoldDiff10Mid,GoldDiff10AD,GoldDiff10Bot,XPDiff10Top,XPDiff10Jg,XPDiff10Mid,XPDiff10AD,XPDiff10Bot,BlueKills10,BlueAssists10,BlueDeaths10,BlueTotalGold10,BlueDragonKills10,BlueTowerKills10,RedKills10,RedAssists10,RedDeaths10,RedTotalGold10,RedDragonKills10,RedTowerKills10,GoldTopBlue15,GoldJgBlue15,GoldMidBlue15,GoldADBlue15,GoldSupBlue15,GoldTopRed15,GoldJgRed15,GoldMidRed15,GoldADRed15,GoldSupRed15,GoldDiff15Top,GoldDiff15Jg,GoldDiff15Mid,GoldDiff15AD,GoldDiff15Bot,XPDiff15Top,XPDiff15Jg,XPDiff15Mid,XPDiff15AD,XPDiff15Bot,BlueKills15,BlueAssists15,BlueDeaths15,BlueTotalGold15,BlueDragonKills15,BlueTowerKills15,RedKills15,RedAssists15,RedDeaths15,RedTotalGold15,RedDragonKills15,RedTowerKills15,BlueInhibKills15,RedInhibKills15,GoldTopBlueEnd,GoldJgBlueEnd,GoldMidBlueEnd,GoldADBlueEnd,GoldSupBlueEnd,GoldTopRedEnd,GoldJgRedEnd,GoldMidRedEnd,GoldADRedEnd,GoldSupRedEnd,GoldDiffEndTop,GoldDiffEndJg,GoldDiffEndMid,GoldDiffEndAD,GoldDiffEndBot,XPDiffEndTop,XPDiffEndJg,XPDiffEndMid,XPDiffEndAD,XPDiffEndBot,BlueKillsEnd,BlueAssistsEnd,BlueDeathsEnd,BlueTotalGoldEnd,BlueDragonKillsEnd,BlueTowerKillsEnd,RedKillsEnd,RedAssistsEnd,RedDeathsEnd,RedTotalGoldEnd,RedDragonKillsEnd,RedTowerKillsEnd,BlueInhibKillsEnd,RedInhibKillsEnd,BlueBaronKillsEnd,RedBaronKillsEnd,VisionScoreTopBlue,VisionScoreJgBlue,VisionScoreMidBlue,VisionScoreADBlue,VisionScoreSupBlue,VisionScoreTopRed,VisionScoreJgRed,VisionScoreMidRed,VisionScoreADRed,VisionScoreSupRed,DamageDealtTopBlue,DamageDealtJgBlue,DamageDealtMidBlue,DamageDealtADBlue,DamageDealtSupBlue,DamageDealtTopRed,DamageDealtJgRed,DamageDealtMidRed,DamageDealtADRed,DamageDealtSupRed,DamageTakenTopBlue,DamageTakenJgBlue,DamageTakenMidBlue,DamageTakenADBlue,DamageTakenSupBlue,DamageTakenTopRed,DamageTakenJgRed,DamageTakenMidRed,DamageTakenADRed,DamageTakenSupRed,TotalCCDurationTopBlue,TotalCCDurationJgBlue,TotalCCDurationMidBlue,TotalCCDurationADBlue,TotalCCDurationSupBlue,TotalCCDurationTopRed,TotalCCDurationJgRed,TotalCCDurationMidRed,TotalCCDurationADRed,TotalCCDurationSupRed,winner
0,2023-06-18 18:14:11.781000+00:00,13.11.512.8126,2068.268,0,2,10,3,10,4,1296.206,1235.483,1067.770,1516.094,1677.850,1698.828,2037.578,2068.268,1716.198,2050.712,2054.181,2068.268,1513.456,1425.611,2068.268,2068.268,1836.628,2068.268,2068.268,2068.268,2068.268,2068.268,1585.0,1503.0,1628.0,1388.0,1604.0,1716.0,2037.0,1564.0,2031.0,1714.0,-131.0,-534.0,64.0,-643.0,-110.0,-338.0,195.0,-61.0,-181.0,-433.0,1.0,0.0,2.0,7708.0,0.0,0.0,2.0,4.0,1.0,9062.0,0.0,0.0,3149.0,2397.0,3364.0,2962.0,2579.0,3720.0,3726.0,2900.0,4183.0,3783.0,-571.0,-1329.0,464.0,-1221.0,-1204.0,-1035.0,-475.0,674.0,36.0,-1222.0,3.0,1.0,9.0,14451.0,0.0,0.0,9.0,10.0,3.0,18312.0,0.0,0.0,0.0,0.0,11282.0,9916.0,12530.0,11370.0,7158.0,15064.0,11988.0,13377.0,12961.0,8681.0,-3782.0,-2072.0,-847.0,-1591.0,-1523.0,-1587.0,-2028.0,-534.0,-1481.0,-1498.0,17.0,29.0,30.0,52256.0,1.0,3.0,30.0,45.0,17.0,62071.0,2.0,11.0,0.0,1.0,0.0,1.0,15.378925,33.785397,29.134930,14.637402,63.788021,19.840324,38.937912,22.034334,31.777031,91.056816,16587.083984,13899.994141,24223.117188,14434.706055,7639.883301,34720.492188,24554.562500,29437.285156,15045.059570,7302.849121,41552.636719,29446.427734,21786.240234,19778.248047,19171.443359,36815.449219,30297.560547,18342.599609,14348.911133,12906.021484,24.019489,11.786745,14.136205,1.084917,

We can reduce the remaining 174 columns by consolidating. For example, we are given the gold of top, jg, mid, AD, and sup at 10 minutes for the blue side and the red side. These can be added into 2 columns. The same can be done for gold differences, vision scores, damage dealt, damage taken, etc. Consolidation will also make modeling easier. There may be nuances to consider, such as possibly keeping the gold of adc separate since esports games favor funneling the adc, especially if more competitive tournaments tend to have longer matches (adc shines late game), but for now we simply want an mvp.

In [ ]:
#df.iloc[19000].to_frame().T to get a row
# nan_rows = df[df[['gameDate']].isna().any(axis=1)] to get rows with NaN in given column

In [46]:
# durbin watson test for autocorrelation, consider time series analysis
# df.columns.tolist()